In [47]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplcyberpunk

plt.style.use("cyberpunk")
import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from tqdm import tqdm
import torchutils as tu
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
torch.manual_seed(666)
import faiss

In [49]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [50]:
df = pd.read_csv("/home/artemiy/recsys_project/data/dataset_new.csv")
df.isna().sum()

url               0
poster            0
title             0
description       6
rating         7751
year              0
genres            1
cast              0
country           0
age               0
dtype: int64

In [51]:
# pip install transformers sentencepiece
# import torch
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")
model.cuda()  # uncomment it if you have a GPU


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(83828, 312, padding_idx=0)
    (position_embeddings): Embedding(2048, 312)
    (token_type_embeddings): Embedding(2, 312)
    (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-2): 3 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=312, out_features=312, bias=True)
            (key): Linear(in_features=312, out_features=312, bias=True)
            (value): Linear(in_features=312, out_features=312, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=312, out_features=312, bias=True)
            (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)

In [52]:
def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()


print(embed_bert_cls("привет мир", model, tokenizer).shape)

(312,)


In [53]:
df.columns

Index(['url', 'poster', 'title', 'description', 'rating', 'year', 'genres',
       'cast', 'country', 'age'],
      dtype='object')

In [54]:
df['description'].isna().sum()

6

In [55]:
# Заполнение пропусков и конвертация в строковый тип
df["description"] = df["description"].fillna("").astype(str)

In [56]:
# Создание эмбеддингов для описаний
description_embeddings = np.vstack(
    df["description"].apply(lambda x: embed_bert_cls(x, model, tokenizer)).values
)

In [57]:
# Построение индекса FAISS
index = faiss.IndexFlatL2(description_embeddings.shape[1])
index.add(description_embeddings.astype(np.float32))

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12972 entries, 0 to 12971
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   url          12972 non-null  object 
 1   poster       12972 non-null  object 
 2   title        12972 non-null  object 
 3   description  12972 non-null  object 
 4   rating       5221 non-null   float64
 5   year         12972 non-null  int64  
 6   genres       12971 non-null  object 
 7   cast         12972 non-null  object 
 8   country      12972 non-null  object 
 9   age          12972 non-null  int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 1013.6+ KB


In [67]:
print(df['country'].unique())

['Россия' 'США' 'США, Канада' 'Великобритания' 'Украина' 'Россия, Куба'
 'СССР' 'Россия, Украина' 'США, Германия, Канада, Франция' 'Бразилия'
 'Испания, Италия' 'Япония' 'Южная Корея'
 'Норвегия, Великобритания, Финляндия, Швеция' 'Канада' 'США, Франция'
 'Турция' 'Норвегия' 'Швеция' 'США, Великобритания' 'Индия'
 'Канада, Великобритания' 'Австралия' 'Беларусь'
 'Бельгия, Германия, Швеция' 'Германия, Дания, Франция, Швеция'
 'Германия, Великобритания, Франция'
 'США, Германия, Великобритания, Швеция' 'Россия, Аргентина'
 'США, Германия, Италия, Канада' 'Китай' 'Россия, Великобритания' 'Италия'
 'Германия' 'США, Япония' 'Канада, Франция' 'Россия, Казахстан, Украина'
 'Беларусь, Украина' 'Бельгия, Германия' 'США, Германия'
 'Россия, Беларусь' 'США, Германия, Испания, Италия, Франция'
 'Италия, Великобритания' 'США, Германия, Канада'
 'Австралия, Великобритания' 'Аргентина' 'Россия, Польша, Украина'
 'Франция' 'Сербия' 'Великобритания, Финляндия'
 'США, Канада, Великобритания' 'Таиланд, Ю

In [59]:
def find_shows(query, df, model, tokenizer, index, top_k=int):
    # Создание эмбеддингов для запроса
    query_embedding = embed_bert_cls(query, model, tokenizer).reshape(1, -1)

    # Поиск топ-K ближайших соседей с использованием Faiss
    _, top_k_indices = index.search(query_embedding.astype(np.float32), top_k)

    # Извлечение соответствующих строк из DataFrame
    result = df.iloc[top_k_indices[0]].copy()

    # Вычисление косинусного сходства для сортировки (опционально)
    similarities = cosine_similarity(
        query_embedding, description_embeddings[top_k_indices[0]]
    ).flatten()
    result["similarity"] = similarities

    return result.sort_values("similarity", ascending=False)

In [60]:
query = "В центре истории сериала оказываются молодые люди, только недавно окончившие сложную образовательную программу в престижной финансовой компании, считающейся одним из мировых лидеров в своей области."
find_shows(query, df, model, tokenizer, index, 10)

,url,poster,title,description,rating,year,genres,cast,country,age,similarity
8599,https://kino.mail.ru/series_913786_zhurnalyugi/,https://resizer.mail.ru/p/0be34b07-2629-5f92-9...,Журналюги,В центре событий сериала «Журналюги» — двое пр...,NaN,2016,"Драма, Детектив","Никита Тарасов, Александр Ляпин, Алексей Барду...",Россия,16,0.766532
10838,https://kino.mail.ru/series_910728_partneri/,https://resizer.mail.ru/p/57fc6ba2-4f2b-5f89-a...,Партнеры,В центре событий комедийного сериала «Партнеры...,NaN,1979,Комедия,"Уилфрид Хайд-Уайт, Мартин Шорт, Элли Миллс, Дж...",США,12,0.751230
4212,https://kino.mail.ru/series_920337_deti/,https://resizer.mail.ru/p/ff954a9b-90be-5f86-a...,Никто не узнает,"Главные герои сериала — успешные, состоявшиеся...",6.9,2022,"Драма, Мелодрама","Юрий Быков, Евгения Брик, Кирилл Сафонов, Викт...",Россия,16,0.743262
5494,https://kino.mail.ru/series_935923_pokolenie_vi/,https://resizer.mail.ru/p/e02be7a2-a53f-55df-8...,Поколение «Ви»,Действие сериала разворачивается в единственно...,5.9,2023,"Фантастика, Комедия, Боевик, Приключения","Джаз Синклер, Патрик Шварценеггер, Лиззи Бродв...",США,18,0.742140
2557,https://kino.mail.ru/series_931231_univer_star...,https://resizer.mail.ru/p/59d883c5-b138-502b-9...,Универ. 10 лет спустя,Продолжение популярного молодежного ситкома «У...,6.6,2021,Комедия,"Арарат Кещян, Станислав Ярушин, Александр Март...",Россия,16,0.739653
6988,https://kino.mail.ru/series_905734_startap/,https://resizer.mail.ru/p/dc977c76-65b1-53bf-a...,Стартап,В центре событий комедийного сериала «Стартап»...,NaN,2015,Комедия,"Парвеш Шина, Питер Шинкода, Джулия Карпентер, ...",США,16,0.739078
3164,https://kino.mail.ru/series_920617_bolshaya_igra/,https://resizer.mail.ru/p/1be6727a-dc55-5deb-9...,Большая игра,Действие фантазийной комедии разворачивается з...,7.2,2018,"Комедия, Спорт","Дмитрий Колчин, Сергей Лавыгин, Максим Лагашки...",Россия,16,0.736978
1187,https://kino.mail.ru/series_913462_proekt_orfei/,https://resizer.mail.ru/p/f7517461-3c97-5f47-9...,Проект «Орфей»,Пятеро лучших студентов-медиков получают уника...,6.7,2016,"Драма, Фантастика, Триллер","Лиза Смит, Стефани ван Лирсум, Тобиас Нироп, В...",Нидерланды,16,0.735264
7438,https://kino.mail.ru/series_925222_industriya/,https://resizer.mail.ru/p/cc620e62-7c2a-5369-8...,Индустрия,Выпускники финансовых школ и институтов вступа...,NaN,2020,Драма,"Эндрю Бакан, Александр Девриент, Джо Херст, Дж...",США,18,0.734215
3669,https://kino.mail.ru/series_836537_kult/,https://resizer.mail.ru/p/4ecb8a1f-f979-55a0-9...,Культ,Несколько молодых ребят из России становятся а...,5.9,2015,"Драма, Приключения","Дмитрий Марьянов, Татьяна Арнтгольц, Артур Соп...",Россия,16,0.731506
